Requirements:
Link Drive to Colab, and have two files: large file called large_corpus.txt, and one called fine_tune_data.txt. Also have a directory where you want all of these to be in

In [ ]:
pip install --upgrade pip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/German_Trainer_LLM')

In [ ]:
!pip install torch torchvision torchaudio
!pip install accelerate==0.20.3
!pip install transformers==4.35.2



  Using cached accelerate-0.20.3-py3-none-any.whl.metadata (17 kB)
Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.1
    Uninstalling accelerate-0.20.1:
      Successfully uninstalled accelerate-0.20.1


  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
Using cached tokenizers-0.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [ ]:
!pip install accelerate==0.20.1 transformers[torch]
!pip install transformers torch


  Using cached accelerate-0.20.1-py3-none-any.whl.metadata (17 kB)
INFO: pip is looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached transformers-4.36.1-py3-none-any.whl.metadata (126 kB)
  Using cached transformers-4.36.0-py3-none-any.whl.metadata (126 kB)
  Using cached transformers-4.35.1-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached transformers-4.34.1-py3-none-any.whl.metadata (121 kB)
  Using cached transformers-4.34.0-py3-none-any.whl.metadata (121 kB)
INFO: pip is still looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
  Using cached 

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./gpt2-finetuned")

# Save the tokenizer
tokenizer.save_pretrained("./gpt2-finetuned")


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

Run this following code to train the model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Assuming 'large_corpus.txt' is your large pre-training dataset
pretrain_dataset = 'large_corpus.txt'

tokenized_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=pretrain_dataset,
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gpt2-pretrained_1",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss
500,3.624100
1000,3.255000
1500,3.086900
2000,2.983100


TrainOutput(global_step=2373, training_loss=3.1918100131196008, metrics={'train_runtime': 422.092, 'train_samples_per_second': 22.488, 'train_steps_per_second': 5.622, 'total_flos': 620045991936000.0, 'train_loss': 3.1918100131196008, 'epoch': 3.0})

In [ ]:
# Assuming 'fine_tune_data.txt' is your smaller fine-tuning dataset
fine_tune_dataset = 'fine_tune_data.txt'

tokenized_fine_tune_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=fine_tune_dataset,
    block_size=128,
)

fine_tune_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

fine_tune_training_args = TrainingArguments(
    output_dir="./gpt2-finetuned_1",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

fine_tune_trainer = Trainer(
    model=model,
    args=fine_tune_training_args,
    train_dataset=tokenized_fine_tune_dataset,
    data_collator=fine_tune_data_collator,
)

fine_tune_trainer.train()

# Save the fine-tuned model
model.save_pretrained("./gpt2-finetuned_1")

# Save the tokenizer
tokenizer.save_pretrained("./gpt2-finetuned_1")



Step,Training Loss
500,2.065500
1000,1.993500
1500,1.932700
2000,1.847500
2500,1.853200
3000,1.771300
3500,1.777300


('./gpt2-finetuned_1/tokenizer_config.json',
 './gpt2-finetuned_1/special_tokens_map.json',
 './gpt2-finetuned_1/vocab.json',
 './gpt2-finetuned_1/merges.txt',
 './gpt2-finetuned_1/added_tokens.json')

Run this Code to test the model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned GPT-2 model and tokenizer
model_path = "./gpt2-finetuned_1/"
model = GPT2LMHeadModel.from_pretrained(model_path, ignore_mismatched_sizes=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_path, use_fast=False, cache_dir=None, ignore_mismatched_sizes=True)

def generate_response(prompt, max_length=50, temperature=0.7):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate response using the fine-tuned GPT-2 model
    output = model.generate(input_ids, max_length=max_length, temperature=temperature, num_return_sequences=1)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

print("Fine-tuned GPT-2 Model Interaction")
print("Type 'exit' to end the conversation.")

while True:
    user_input = input("You: ")

    if user_input.lower() == 'exit':
        print("Exiting the conversation.")
        break

    model_response = generate_response(user_input)
    print("Model:", model_response)


Fine-tuned GPT-2 Model Interaction
Type 'exit' to end the conversation.
You: Wie Gehts?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model: Wie Gehts?
- Ich bin ein Rechtsanwalt.
- Wie geht es dir?
- Ich bin ein Rechtsanwalt.

Prompt: Wie geht es
You: Hallo!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model: Hallo!
- Ich bin ein Rechtsanwalt.
- Ich bin ein Rechtsanwalt.
- Ich bin ein Rechtsanwalt.
- Ich bin ein Rech


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 20>:21                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py:851 in raw_input                 │
│                                                                                                  │
│   848 │   │   │   raise StdinNotImplementedError(                                                │
│   849 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   850 │   │   │   )                                                                              │
│ ❱ 851 │   │   return self._input_request(str(prompt),                                            │
│   852 │   │   │   self._parent_ident,                                                            │
│   853 │   │   │   self._parent_header,                                                           │
│   854 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py:895 in _input_request            │
│                                                                                                  │
│   892 │   │   │   │   │   │   break                                                              │
│   893 │   │   │   except KeyboardInterrupt:                                                      │
│   894 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 895 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   896 │   │   │   except Exception as e:                                                         │
│   897 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   898                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user